In [1]:
import os

from pysld.style import StyleSld

import seabeepy as sb

## 1. Simple styles

In [2]:
out_fold = r"/home/notebook/annotation/sld_files"

In [3]:
# Dict mapping file names to fill and stroke colours
# name => (fill_colour, stroke_colour)
style_dict = {
    "black_outline": (None, "#000000"),
    "red_outline": (None, "#FF0000"),
    "blue_outline": (None, "#0000FF"),
}
for name, (fill, stroke) in style_dict.items():
    sld = StyleSld(
        style_name="polygonStyle",
        geom_type="polygon",
        fill_color=fill,
        stroke_color=stroke,
    )
    sld_style = sld.generate_simple_style()
    sld_path = os.path.join(out_fold, f"{name}.sld")
    with open(sld_path, "w") as f:
        f.write(sld_style)

In [4]:
versions = ["0-1", "0-2", "0-4", "1-0"]
for version in versions:
    df = sb.anno.get_class_codes(version)
    del df["desc"]
    for level in [1, 2, 3]:
        code_len = level * 2
        lev_df = df[df["code"].str.len() == code_len].drop_duplicates()

        sld = StyleSld(
            style_name="polygonStyle",
            geom_type="polygon",
            attribute_name=f"lev{level}_name",
            values=lev_df["name"].tolist(),
            color_palette=lev_df["colour"].tolist(),
            opacity=0.5,
        )
        sld_style = sld.generate_categorized_style()
        sld_path = os.path.join(
            out_fold, f"annotation_classes_v{version}_level{level}.sld"
        )
        with open(sld_path, "w") as f:
            f.write(sld_style)